In [11]:
!pip install openai==0.28


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
from datasets import load_dataset 
from evaluator_fast import Evaluate
from consistency_checker_fast import SemanticConsistnecyCheck
import time
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

qa_data = load_dataset("json", data_files="dataset/hotpotQA_halu.json")

In [13]:
def hallucination_score(question, target_answer, model, num_samples):
    
    # llm evaluation
    llm_evaluate = Evaluate(model=model)
    scc = SemanticConsistnecyCheck(model=model)
    # fast self-evaluation
    fast_self_responses = llm_evaluate.self_evaluate_api(self_question = question, temperature = 1.0, self_num = num_samples)
    # fast consistency checker 
    fast_consistency_res = scc.score_scc_api(question, target_answer, candidate_answers = fast_self_responses, temperature = 0.0)
    
    return fast_consistency_res[0]

In [14]:
# fastsac3 hallucation scores

n_data = 10
model = 'gpt-3.5-turbo'
num_samples_list = [3,5,10,15]

for num_samples in num_samples_list:
    
    t0 = time.time()
    halu_score_all = []
    filename = 'halu_fastsac3_' + str(n_data) + '_' + str(num_samples) + '.txt'

    for i in tqdm(range(n_data)):
        if i <= n_data // 2:
            target_answer = qa_data['train'][i]['hallucinated_answer']
        else:
            target_answer = qa_data['train'][i]['right_answer']

        question = qa_data['train'][i]['question']
        halu_score = hallucination_score(question, target_answer, model, num_samples)
        halu_score_all.append(halu_score)

    # auroc 
    print('Time per query', (time.time()-t0)/n_data)
    true_label = [1]*(n_data // 2) + [0] * (n_data // 2)
    roc_auc = roc_auc_score(true_label,halu_score_all)
    print('AUROC score', roc_auc)
        

100%|███████████████████████████████████████████| 10/10 [00:17<00:00,  1.78s/it]


Time per query 1.7763252973556518
AUROC score 0.62


100%|███████████████████████████████████████████| 10/10 [00:19<00:00,  1.97s/it]


Time per query 1.965973401069641
AUROC score 0.8


100%|███████████████████████████████████████████| 10/10 [00:20<00:00,  2.02s/it]


Time per query 2.016736102104187
AUROC score 0.68


100%|███████████████████████████████████████████| 10/10 [00:21<00:00,  2.16s/it]

Time per query 2.1567198038101196
AUROC score 0.74
